# a)

# First method : Using article below
# Referance : https://arxiv.org/pdf/2406.14252

In [1]:
import numpy as np
import strawberryfields as sf
from strawberryfields.ops import *
import random

np.random.seed(1)
# Number of modes/spins
N = 10

# Ising Couplings
J = {(i, j): np.random.uniform(-1, 1) for i in range(1, N) for j in range(i + 1, N + 1)}

# Create a unitary matrix 
random_matrix = np.random.rand(N, N)
unitary_matrix, _ = np.linalg.qr(random_matrix)

# Improved parameter initialization based on the paper
params = np.random.uniform(-0.1, 0.1, N)

# Define a GBS circuit
def ising_circuit(params):
    prog = sf.Program(N)
    with prog.context as q:
        for i in range(N):
            Sgate(params[i]) | q[i]
        Interferometer(unitary_matrix) | q
    return prog

# Define the cost function
def cost(params):
    prog = ising_circuit(params)
    eng = sf.Engine("gaussian")
    result = eng.run(prog)
    state = result.state

    # Compute the expectation value of the Hamiltonian
    means = state.means()[:N]
    covariance = state.cov()[:N, : N]
    H_expval = 0

    for (i, j), value in J.items():
        H_expval += value * (means[i-1] * means[j-1] + covariance[i-1, j-1])

    # Regularization term to prevent large parameter values
    reg_term = np.sum(params**2)
    
    return H_expval + 0.01 * reg_term

# Optimization
from scipy.optimize import minimize

# Minimize the cost function to find the ground state energy with bounds to prevent large values
bounds = [(-1, 1) for _ in range(N)]
result = minimize(cost, params, method='L-BFGS-B', bounds=bounds)

# Optimized parameters and ground state energy
optimized_params = result.x
ground_state_energy = result.fun

print(f"Ground state configuration: {optimized_params}")
print(f"Ground state energy: {ground_state_energy}")


Ground state configuration: [ 1. -1.  1.  1. -1. -1.  1. -1. -1.  1.]
Ground state energy: -17.327640689895624


# Second method: Training variational GBS distributions
# Reference1 : https://strawberryfields.ai/photonics/apps/run_tutorial_training.html#id3
# Reference2 : https://arxiv.org/pdf/2004.04770

In [ ]:
import networkx as nx
from strawberryfields.apps import train

np.random.seed(1)

# Number of modes/spins
N = 10

# Ising couplings
J = {(i, j): np.random.uniform(-1, 1) for i in range(N) for j in range(i + 1, N)}

# Create a complete graph 
graph = nx.complete_graph(N)
A = nx.to_numpy_array(graph)

# Define the Ising Hamiltonian function
def Ising_Hamiltonian(s):
    H = 0
    for (i, j), value in J.items():
        H += -value * s[i] * s[j]
    return H

# Initialize weights for the VGBS model
weights = train.Exp(N)

# Mean photon number
n_mean = 10

# Create a VGBS object
vgbs = train.VGBS(A, n_mean, weights, threshold=False)

# Define the cost function using stochastic sampling
cost = train.Stochastic(Ising_Hamiltonian, vgbs)

# Initialize parameters
params = np.zeros(N)
nr_samples = 600
nr_steps = 700
rate = 0.001

print('Initial mean photon numbers = ', vgbs.mean_photons_by_mode(params))

# Perform the gradient-based optimization
for i in range(nr_steps):
    params -= rate * cost.grad(params, nr_samples)
    if i % 50 == 0:
        print(f'Cost = {cost.evaluate(params, nr_samples):.3f}')
        print('Mean photon numbers = ', vgbs.mean_photons_by_mode(params))
        
print(f'Final optimized parameters: {params}')

Initial mean photon numbers =  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Cost = 2.897
Mean photon numbers =  [0.93110128 0.93397769 0.93488418 0.93398848 0.9341467  0.93237986
 0.93589346 0.93451067 0.9343511  0.93562841]
Cost = 0.654
Mean photon numbers =  [0.29467226 0.31856384 0.328631   0.31769131 0.32238362 0.3051944
 0.33683732 0.32432887 0.32026531 0.33308132]
Cost = 0.357
Mean photon numbers =  [0.22548068 0.25194261 0.26449133 0.25069275 0.25739078 0.23703785
 0.27364047 0.25922891 0.25344212 0.26877573]
Cost = 0.239
Mean photon numbers =  [0.19514101 0.2232117  0.23759367 0.22174517 0.22987231 0.20729239
 0.24743692 0.23169003 0.22461858 0.24176793]
Cost = 0.173
Mean photon numbers =  [0.17719546 0.20658171 0.22255891 0.2049661  0.21427811 0.18980371
 0.23304394 0.21613203 0.2079504  0.22667921]
Cost = 0.130
Mean photon numbers =  [0.16508024 0.19566292 0.21313324 0.19393413 0.20430578 0.17808669
 0.22425558 0.20623537 0.19702325 0.21723767]
Cost = 0.097
Mean photon numbers =  [0.15629

In [ ]:
# Generate samples 
Aw = vgbs.A(params)
samples = vgbs.generate_samples(Aw, n_samples=200)

print("States:")
print(samples)

In [ ]:
# Map photon numbers to Ising (-1 or 1)
def photon_to_spin(photon_counts):
    return [1 if count > 0 else -1 for count in photon_counts]

# Calculate the ground state energy and configuration
sample_energies = np.array([Ising_Hamiltonian(photon_to_spin(sample)) for sample in samples])
ground_state_energy = sample_energies.min()
ground_state = samples[sample_energies.argmin()]

# Convert the ground state photon counts to Ising spins
ground_state_spins = photon_to_spin(ground_state)

print(f'Ground state configuration: {ground_state_spins}')
print(f'Ground state energy: {ground_state_energy}')

# b)

In [ ]:
from dimod import ExactSolver

np.random.seed(1)
# N = Number of spins
N = 10
h = {}
J = {}
J = {(i, j): np.random.uniform(-1, 1) for i in range(1,N) for j in range(i + 1, N + 1)}

sampler = ExactSolver()

samplest = sampler.sample_ising(h, J, num_reads=1000)
print("Exact solution :")
print(samplest)

# c)

In [ ]:
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite

np.random.seed(1)
# N = Number of spins
N = 10
h = {}
J = {}
J = {(i, j): np.random.uniform(-1, 1) for i in range(1,N) for j in range(i + 1, N + 1)}

sampler = EmbeddingComposite(DWaveSampler(token="DEV-91deb9e83ca219cded0c79faef452cf24823a325"))

samplest = sampler.sample_ising(h, J, num_reads=1000)

print("D-Wave solution :")
print(samplest)